In [8]:
#!pip install -r requirements.txt

In [2]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

#load environment variables
load_dotenv()


True

In [3]:
chatmodel = ChatOpenAI(model="gpt-3.5-turbo")

In [ ]:
chat_result = chatmodel.invoke("What is special about India?")

In [ ]:
# To get only cotents of the result. It only contains the output message
print("content only: ", chat_result.content)

# To get full message from the chat model
# It contains details of the prompt, model, prompt tokens, result tokens
print("full result: ", chat_result)

# **Examples for basic conversation using langchain and openai model**

In [9]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

# defining messages with its proper type
messages = [
    # setting the chatmodel behaviour for the response of the query
    # It always come first, as it sets the context.
    SystemMessage(content="You are an expert assistant."),
    # setting the real query in message form
    HumanMessage(content="Who won the world series in 2020?")
]

In [ ]:
conv_result = chatmodel.invoke(messages)
print("result message: ", conv_result.content)

In [10]:
# Now, inorder to keep the conversation going on for longer context window.
PREV_CONTEXT_LIMIT = 10

def get_conversation_result(messages, humanMessage, prevAIMessage):
  if mCount >= PREV_CONTEXT_LIMIT:
    mCount = 0
    print("context limit crossed. Starting conversation with new context")
    while len(messages)>1:
      messages.pop(0)
    messages.append(humanMessage)
    return AIMessage(content=chatmodel.invoke(messages).content), messages
  else:
    messages.append(prevAIMessage)
    messages.append(humanMessage)
    return AIMessage(content=chatmodel.invoke(messages).content), messages



In [ ]:
# Running the loop of conversation for messages

prevAIMessage = AIMessage(content=chatmodel.invoke(messages).content)

while True:
  query = input(" User: ")
  if query.lower() == "exit":
    break
  # get human message from user query only for each coversation loop as a form of input
  humanMessage = HumanMessage(content=query)

  # get chat result as well as updated messages
  prevAIMessage, messages = get_conversation_result(messages, humanMessage, prevAIMessage)



# **Understanding the chat Prompt template for building chatbots**

In [ ]:
# ChatPrompt template helps in creating custom chat query with parameters.
# It can be seen as masked prompt template.
from langchain.prompts import ChatPromptTemplate

chatTemplate = "Explain this {topic} in details"
prompt_template = ChatPromptTemplate.from_template(chatTemplate)

prompt = prompt_template.invoke({"topic": "Machine Learning"})

chatResult = chatmodel.invoke([prompt])
print(chatResult.content)

In [ ]:
# We can also change the context of the messages on runtime(dynamically)
# Like changing the context of the SystemAI message

advancedTemplate = ChatPromptTemplate.format_messages(
    [
        ("system", "You are an expert in the {topic}."),
        ("human", "Please explain me in details about the {subject} along with its pros & cons."),
    ]
)

advancedPrompt = advancedTemplate.invoke({"topic": "Machine Learning", "subject": "Decision Trees"})

advancedResult = chatmodel.invoke(advancedPrompt)
print(advancedResult.content)